In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, normalize


In [2]:
G = np.load('MERFISH_AIBS10xNN.npz', allow_pickle=True)['G']
D = np.load('MERFISH_AIBS10xNN.npz', allow_pickle=True)['D']
aibs_cell = np.load('MERFISH_AIBS10xNN.npz', allow_pickle=True)['aibs_cell']
merfish_cell = np.load('MERFISH_AIBS10xNN.npz', allow_pickle=True)['merfish_cell']
labelref = pd.read_csv('/home/jzhou_salk_edu/sky_workdir/230214_merfish_integration/RNA_cocluster.csv', header=0, index_col=0)['leiden']
labelref = labelref.loc[aibs_cell]


In [3]:
chunk_size = 50000
sd = 1
start_time = time.time()
cellfilter = (D[:,-1]==0)
D = (1 - D/D[:,-1][:,None])
D[cellfilter] = 1
D = 1 - np.exp(-D*(sd**2)/4)
D = D / (np.sum(D, axis=1) + 1e-6)[:,None]
print(time.time() - start_time)
enc = OneHotEncoder()
# enc.fit(adata_merge.obs.loc[aibs_cell, ['L3']].values.astype(str))
labelref = enc.fit_transform(labelref.loc[aibs_cell][:,None])
cluster = pd.DataFrame(index=merfish_cell, columns=['cocluster', 'score'], dtype=str)
if len(enc.categories_[0])==1:
    cluster['cocluster'] = enc.categories_[0][0]
    cluster['score'] = 1
else:
    for chunk_start in range(0, len(merfish_cell), chunk_size):
        result = (D[chunk_start : (chunk_start + chunk_size), :, None] * labelref[G[chunk_start : (chunk_start + chunk_size)].flatten()].toarray().reshape((-1, min(25, len(aibs_cell)), enc.categories_[0].shape[0]))).sum(axis=1)
        result = pd.DataFrame(result, columns=enc.categories_[0], index=merfish_cell[chunk_start : (chunk_start + chunk_size)])
        result = result.loc[:, result.columns!='nan']
        cluster.loc[result.index, 'cocluster'] = result.idxmax(axis=1).values
        cluster.loc[result.index, 'score'] = result.max(axis=1).values
        print(chunk_start)
    
print(time.time() - start_time)
cluster.to_hdf('MERFISH_cocluster.hdf', key='data')


0.08375120162963867
0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
35.256422996520996
